In [1]:
# -*- coding: utf-8 -*-

import numpy as np
np.random.seed(111)
import argparse
import os
import json
from yolo.frontend import create_yolo, get_object_labels

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"


def setup_training(config_file):
    """make directory to save weights & its configuration """
    import shutil
    with open(config_file) as config_buffer:
        config = json.loads(config_buffer.read())
    dirname = config['train']['saved_folder']
    if os.path.isdir(dirname):
        print("{} is already exists. Weight file in directory will be overwritten".format(dirname))
    else:
        print("{} is created.".format(dirname, dirname))
        os.makedirs(dirname)
    print("Weight file and Config file will be saved in \"{}\"".format(dirname))
    shutil.copyfile(config_file, os.path.join(dirname, "config.json"))
    return config, os.path.join(dirname, "weights.h5")



C:\Users\ER9X\Anaconda3\envs\yolo\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\ER9X\Anaconda3\envs\yolo\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\ER9X\Anaconda3\envs\yolo\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\ER9X\Anaconda3\envs\yolo\lib\site-packages\tensorflow\python\framework

In [2]:
config, weight_file = setup_training("config.json")

if config['train']['is_only_detect']:
    labels = ["object"]
else:
    if config['model']['labels']:
        labels = config['model']['labels']
    else:
        labels = get_object_labels(config['train']['train_annot_folder'])
print(labels)

# 1. Construct the model 
yolo = create_yolo(config['model']['architecture'],
                   labels,
                   config['model']['input_size'],
                   config['model']['anchors'],
                   config['model']['coord_scale'],
                   config['model']['class_scale'],
                   config['model']['object_scale'],
                   config['model']['no_object_scale'])

# 2. Load the pretrained weights (if any) 
yolo.load_weights(config['pretrained']['full'], by_name=True)

# 3. actual training 



lane is already exists. Weight file in directory will be overwritten
Weight file and Config file will be saved in "lane"
['car', 'human', 'left', 'right', 'yow']
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
mobilenet_0.75_224 (Model)   (None, 7, 7, 768)         1832976   
_________________________________________________________________
detection_layer_50 (Conv2D)  (None, 7, 7, 50)          38450     
_________________________________________________________________
reshape_1 (Reshape)          (None, 7, 7, 5, 10)       0         
Total params: 1,871,426
Trainable params: 1,855,010
Non-trainable params: 16,416
_________________________________________________________________
Loading pre-trained weights in model_classification2.h5


In [3]:

yolo.train(config['train']['train_image_folder'],
           config['train']['train_annot_folder'],
           config['train']['actual_epoch'],
           weight_file,
           config["train"]["batch_size"],
           config["train"]["jitter"],
           config['train']['learning_rate'], 
           config['train']['train_times'],
           config['train']['valid_times'],
           config['train']['valid_image_folder'],
           config['train']['valid_annot_folder'],
           config['train']['first_trainable_layer'],
           config['train']['is_only_detect'])
# loss: 2.1691, train batch jitter=False

Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/50
241/241 [==============================] - 35s 145ms/step - loss: 1.8421 - val_loss: 0.2264
Epoch 2/50
241/241 [==============================] - 30s 124ms/step - loss: 1.3127 - val_loss: 0.2430
Epoch 3/50
241/241 [==============================] - 30s 124ms/step - loss: 1.3606 - val_loss: 0.2000
Epoch 4/50
241/241 [==============================] - 30s 124ms/step - loss: 1.4890 - val_loss: 0.1806
Epoch 5/50
241/241 [==============================] - 30s 125ms/step - loss: 1.5585 - val_loss: 2.1919
Epoch 6/50
241/241 [==============================] - 31s 127ms/step - loss: 1.1527 - val_loss: 0.1946
Epoch 7/50
241/241 [==============================] - 31s 127ms/step - loss: 1.2608 - val_loss: 1.9021
Epoch 8/50
241/241 [==============================] - 31s 127ms/step - loss: 1.2287 - val_loss: 1.0130
Epoch 9/50
241/241 [===========